# 导入模块

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns 
#sns.set(style="darkgrid")
plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False


from dateutil.relativedelta import relativedelta
from datetime import datetime as dt
import time

import warnings  
warnings.filterwarnings("ignore")

import sys
import pymysql
from sqlalchemy import create_engine
import json
import datetime 
from datetime import timedelta ,time 
import re

In [2]:
pd.set_option('display.max_columns', None)

# 定义数据库连接

In [3]:
def query(sql,
          host="rm-wz930e5269fur1ht1mo.mysql.rds.aliyuncs.com",
          user="wxz",
          password="5JRcY9SaiepVlIq7iuPo",
          database = '' ,
          port=3306
         ):   
    conn = pymysql.connect(
        host=host,
        user=user,
        port=port,
        password=password,
        max_allowed_packet=1073741824,
        charset="utf8")
    try:
        df = pd.read_sql(sql, con=conn)
        conn.close()
    except:
        print('error')
        conn.close()
        raise
    return df

In [4]:
sql1 = ''' -- 订单&风控信息  近10日数据   
SELECT date(om.create_time) as create_date,om.create_time,om.id as order_id ,om.order_number,om.all_money 
,om.status
,case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as status2 
,case when locate('租物',pa.name)>0 or locate('租物',cc.name)>0 or locate('芝麻',pa.name)>0 or locate('芝麻',cc.name)>0  then '芝麻租物' when locate('抖音',pa.name)>0 then '抖音渠道' when locate('搜索',cc.name)>0 then '搜索渠道' else '其他渠道' end as channel_type 
,tod.sku_attributes,tod.product_name,tod.new_actual_money
,case when  locate('租完即送',tod.sku_attributes)>0 then '租完即送' else '租物归还' end as back_type
,om.user_mobile,tmu.true_name,tmu.id_card_num
,top.total_describes,tor.decision_result,om.cancel_reason
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.traceid') end,'"','') as trace_id 
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.rejected') end,'"','') as rejected 
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.result') end,'"','') as result 
,cc.name as channel_name         -- 来源渠道
,pa.name as activity_name        -- 活动名称
,om.merchant_id,om.merchant_name
,topay.total_freeze_fund_amount 
,om.buy_service_product,tso.status as service_status 
,om.order_method
from  db_digua_business.t_order  om 
left join db_digua_business.t_order_risk tor on om.id = tor.order_id
 -- 备注信息合并 
left join ( SELECT  t.order_id,JSON_ARRAYAGG(t.describes) as total_describes from db_digua_business.t_order_personnel t   GROUP BY 1 ) top 
on om.id = top.order_id 
-- 服务信息
left join  db_digua_business.t_service_order tso  on om.id = tso.order_id 
-- 渠道名称
left join db_digua_business.t_channel cc on om.channel = cc.scene 
-- 活动名称
left join db_digua_business.t_platform_activity pa on om.activity_id = pa.id
-- 用户信息 
left join db_digua_business.t_member_user tmu on om.user_id = tmu.id
-- 商品信息
left join db_digua_business.t_order_details tod on om.id = tod.order_id
-- 免押信息  
left join (SELECT t.*,row_number() over(partition by t.order_id order by t.pay_date desc) as rn 
from db_digua_business.t_order_pay t 
where t.pay_type = 'ZFBYSQ' and t.item_type=1 and t.`status` in (2,5) and t.trade_no is not null )  topay 
on topay.order_id=om.id   and  topay.rn = 1 
where om.user_mobile is not null 
and tmu.true_name not in ("刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华"
,"邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文",'张娜','罗文龙','孔靖','彭康力','何薪华','夏玥','潘佳','包闻天','方全龙','李楠','向圆圆','黄兰娟','林婉婷','廖丽敏','李巧玲','李巧凤','刘三妹','蔡斯静'
,'陈宜诗','陈宝易','林寅钗','谢金凤','刘宏生','骆昌鑫','何静')
-- and COALESCE(pa.name, '未知') not in ("1000单秘密计划","1000单秘密计划-无优惠","1000单曙光计划","线下门店3个月试行") 
and  om.create_time >= DATE_ADD(CURRENT_DATE,INTERVAL -15 day )
-- and  om.create_time <= CURRENT_DATE   -- 近一天数据
-- and  om.create_time <= '2024-10-12 18:30:00'
-- and  om.create_time >= DATE_ADD(CURRENT_DATE,INTERVAL -14 day )
-- and  om.create_time <= DATE_ADD(CURRENT_DATE,INTERVAL -0 day )
;
'''

df_order = query(sql1)
df_order.shape

(23746, 29)

# 合并数据

In [5]:
df = df_order.copy()
df.order_id.notnull().sum()

np.int64(23746)

# 处理日期

In [6]:
df["下单日期"]=pd.to_datetime(df["create_time"]).dt.date
df["下单日期"]=pd.to_datetime(df["下单日期"],errors="coerce")
df["下单日期"]
df["月份"]=df["下单日期"].dt.month
df.shape

(23746, 31)

In [7]:
df['拒绝理由'] = df["rejected"].str.replace("[","").str.replace("]","").str.replace('"','')
df['拒绝理由'].value_counts()

拒绝理由
                                                                                                                       11385
冰鉴火眸分<500命中强拒                                                                                                           3947
命中强拒加入临时黑名单                                                                                                             1406
年龄超过49岁或低于18岁                                                                                                            713
命中自有模型强拒                                                                                                                 500
                                                                                                                       ...  
命中云商分低于496强拒, 命中策略240703_14强拒, 命中策略240703_4强拒, 命中自有模型强拒, 命中蚁盾分>50, 命中融安分<700, 命中融安分低于680强拒, 命中青云分<500, 评分等级低于D,命中拒绝        1
命中策略240703_4强拒, 命中融安分低于680强拒, 评分等级低于D,命中拒绝                                                                              

# 处理备注信息

In [8]:
df = df[df['sku_attributes'].notnull()]

In [9]:

df["取消原因"]=df["total_describes"].str.split("客户申请取消：").str[1].str.split("$").str[0].str.strip()
df["电审拒绝原因"]=df["total_describes"].str.split("审核不通过：").str[1].str.split("$").str[0].str.strip()
df["无法联系原因"]=df["total_describes"].str.split("用户无法联系：").str[1].astype(str).str.split("$").str[0].str.strip()
df.loc[:,"商品类型"]=np.where(df["product_name"].str.contains('99新')|df["product_name"].str.contains('95新')|df["product_name"].str.contains('准新'),"二手","全新")  ##  准新
df.loc[:,"租赁方案"]=np.where(df["sku_attributes"].str.contains('租完即送'),"租完即送","租完归还")
df.loc[:,"押金类型"]=np.where(df["total_freeze_fund_amount"]>0,"部分免押","全免押")
df.loc[:,"优惠券使用否"]=np.where((df["new_actual_money"]-df["all_money"]>0)&(df["租赁方案"]=='租完即送'),"已使用","未使用")
df.loc[:,"是否进行预授权"]=np.where(df["total_freeze_fund_amount"].isnull(),"未预授权","已预授权")

def getcolor(s):
    color_list=json.loads(s)
    for j in range(0,len(color_list)):
        if color_list[j]["key"]=="颜色":
            return color_list[j]["value"]
df.loc[:,"颜色"]=df.apply(lambda x:getcolor(x["sku_attributes"]),axis=1)
df["颜色"].value_counts()  

# %%
def getneicun(s):
    color_list=json.loads(s)
    for j in range(0,len(color_list)):
        if color_list[j]["key"]=="内存":
            return color_list[j]["value"]
df.loc[:,"内存"]=df.apply(lambda x:getneicun(x["sku_attributes"]),axis=1)
df["内存"].value_counts() 

df.loc[:,"进件"]=np.where((df["status2"]=="待支付")|(df["status2"]=="订单取消"),"未进件","进件")
df["进件"].value_counts()

# %%
df.loc[:,"来源渠道"]=df["channel_name"].fillna("未知渠道")

In [10]:
df['免审'] = np.where(df.decision_result.str.contains(pat='免人审',regex=False),1,0)
df['免审'].value_counts()

免审
0    22800
1      946
Name: count, dtype: int64

# 归属渠道

In [11]:
def qudao_type(a,b,c):
    a = str(a)
    b = str(b)
    if "租物" in b:
        return "芝麻租物"
    elif "芝麻" in b:
        return "芝麻租物"
    elif "抖音" in b:
        return "抖音渠道"
    elif "搜索" in a:
        return "搜索渠道"
    elif "租物" in a:
        return "芝麻租物"
    elif "芝麻" in a:
        return "芝麻租物"
    elif c == 1:
        return "芝麻租物"    
    elif "叮咚直播" in a:
        return "叮咚直播"
    elif "租瓜直播2号" in a:
        return "租瓜直播2号"
    elif "租瓜直播" in a:
        return "租瓜直播"
    elif "直播" in a:
        return "小舞直播"
    elif "繁星" in a:
        return "繁星"
    elif "生活号" in a:
        return "生活号"
    elif "群" in a:
        return "支付宝社群"
    elif "付费灯火" in a:
        return "付费灯火"
    else :
        return a
    

# %%
df.loc[:,"归属渠道"]=df.apply(lambda x:qudao_type(x["来源渠道"],x["activity_name"],x["order_method"]),axis=1)
df["归属渠道"].value_counts()

归属渠道
芝麻租物                              13611
搜索渠道                               6674
单人聊天会话中的小程序消息卡片（分享）                1208
我的小程序入口                             394
未知渠道                                278
小程序商家消息（服务提醒）                       261
其他渠道场景渠道。                           244
九州信息                                219
付费流量(通过商家数字推广平台，灯火等投放的广告)           174
抖音渠道                                157
支付宝社群                               126
付费侠客行-苹果旗舰                           80
订阅消息                                 76
生活号                                  64
支付宝 push 消息（同1014）                   56
小舞直播                                 38
小程序快窗                                17
广州易合顺                                16
侠客行vivo                              15
支付宝客户端首页                             10
支付成功页                                 8
第三方 APP（如钉钉）打开，在跳转链接中传入访问来源参数：        6
乐潼月活LT05                              6
乐潼月活LT07                              2
支付宝学生特惠小程序                         

# 订单去重

In [12]:
dict_status_code = {
    "订单取消": 1,
    "待支付": 2,
    "已退款": 3, 
    "待审核": 4,
    "待发货": 5,
    "待收货": 6,
    "租赁中": 7,
    "已完成": 8
}
df["状态编码"]=df["status2"].map(dict_status_code)


df.sort_values(by=["下单日期","状态编码"],inplace=True)


#删除重复单号
df.drop_duplicates(subset=["order_id"],inplace=True)
#删除身份证空值行
df.dropna(subset=["id_card_num"],axis=0,inplace=True)
#去刷单订单
df.drop(df[df['total_describes'].str.contains(pat='panli',regex=False)==True].index,inplace=True)
df.drop(df[df['activity_name']=="1000单秘密计划"].index,inplace=True)
df.drop(df[df['activity_name']=="1000单秘密计划-无优惠"].index,inplace=True)
df.drop(df[df['activity_name']=="1000单曙光计划"].index,inplace=True)
df.drop(df[df['activity_name']=="线下门店3个月试行"].index,inplace=True)
#删除身份证空值行
df.dropna(subset=["status2"],axis=0,inplace=True)


#删除重复订单
df.drop_duplicates(subset=["order_id"],inplace=True)
df.drop_duplicates(subset=["true_name","user_mobile","id_card_num","下单日期"],keep="last",inplace=True)


df.drop(df[df['true_name'].isin(["刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华"
,"邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文"])].index,inplace=True)

# 定义状态

In [13]:
# 判断 前置拦截   机审强拒   出库前风控强拒
df['是否前置拦截'] = np.where((df.result.str.contains('id_card不得为空') )|(df.result.str.contains('mobile校验不通过')) |(df.result.str.contains('name校验不通过'))
                        |(df.result.str.contains('年龄超过49岁或低于18岁'))|(df.result=='风控未通过')|(df.result=='黑名单用户'),1,0)
df['是否机审强拒'] = np.where((df.result.str.contains('风控拒绝') & (~df.result.str.contains('命中出库前风控流强拒').fillna(False)) ),1,0)
df['是否出库前风控强拒'] = np.where((df.result.str.contains('命中出库前风控流强拒').fillna(False)) |((df.total_describes.str.contains('蚂蚁数控风险等级').fillna(False)) & (~df.result.str.contains('黑名单用户').fillna(False))) ,1,0)


In [14]:
def reject_type(a,b,c,d,e,f,g,h,i,j):
    if h==1 and b=="未进件":
        return "前置拦截"
    elif i==1 and b=="进件":
        return "机审强拒"    
    #抖音渠道
    # elif h==1 and b=="进件":
    #     return "机审强拒"  
    elif  b=="进件" and c is not np.nan:
        return "人审拒绝"
    elif  b=="进件" and d is not np.nan:
        return "客户取消"
    elif  b=="进件" and f is not np.nan:
        return "无法联系"
    elif  b=="进件" and j==1 :
        return "出库前风控强拒"
    elif  b=="进件" and "待审核" in e:
        return "待审核"
    elif  b=="进件" and "待发货" in e:
        return "出库"
    elif  b=="进件" and "待收货" in e:
        return "出库"
    elif b=="进件" and "租赁中" in e:
        return "出库"
    elif  b=="进件" and "已完成" in e:
        return "出库"
    elif a is np.nan and b=="未进件":
        return "未进件"
    elif  b=="未进件":
        return "进件前取消"
    else :
        return e

df.loc[:,"审核状态"]=df.apply(lambda x: reject_type(x["拒绝理由"],x["进件"],x["电审拒绝原因"],x["取消原因"],x["status2"],x["无法联系原因"],x["total_describes"],x['是否前置拦截'],x['是否机审强拒'],x['是否出库前风控强拒']),axis=1)#,x["无法联系原因"]
df["审核状态"].value_counts()  

审核状态
进件前取消    8638
前置拦截     4158
机审强拒     1895
无法联系     1215
客户取消      514
人审拒绝      494
Name: count, dtype: int64

# 保留商家信息

In [15]:
df_contain = df.copy()

# 剔除商家数据

In [16]:
#剔除商家数据只保留自营租机业务数据
df.drop(df[df['merchant_name']=="深圳优优大数据科技有限公司"].index,inplace=True)
df.drop(df[df['merchant_name']=="优优2店"].index,inplace=True)
df.drop(df[df['merchant_name']=="小豚租（代收）"].index,inplace=True)
df.drop(df[df['merchant_name']=="苏州蚁诺宝"].index,inplace=True)
df.drop(df[df['merchant_name']=="租着用电脑数码"].index,inplace=True)
df.drop(df[df['merchant_name']=="北京海鸟窝科技有限公司"].index,inplace=True)

df.drop(df[df['merchant_name']=="汇客好租"].index,inplace=True)
df.drop(df[df['merchant_name']=="澄心优租"].index,inplace=True)
df.drop(df[df['merchant_name']=="CPS渠道合作"].index,inplace=True)
df.drop(df[df['sku_attributes'].str.contains(pat='探路者',regex=False)==True].index,inplace=True)  
# 趣智数码  单
df.drop(df[df['merchant_name']=="趣智数码"].index,inplace=True)
df.drop(df[df['merchant_name']=="格木木二奢名品"].index,inplace=True)
df.drop(df[df['merchant_name']=="广州康基贸易有限公司"].index,inplace=True)
# df.drop(df[df['merchant_name']=="汇客好租"].index,inplace=True)


# 去重订单明细导出

In [17]:
from datetime import datetime  
today = datetime.today() 
t_date = today.date().strftime('%Y-%m-%d')
t_date

'2025-02-10'

# 导入订单服务

In [18]:
df["下单时段"]=df["create_time"].astype(str).str[:14]

# %%
df_j = df[df["进件"]=="进件"]
df_j.shape

(3827, 52)

In [19]:
df["待审核"] = np.where(df["审核状态"]=='待审核',1,0)
df["前置拦截"] = np.where(df["审核状态"]=='前置拦截',1,0)
df["机审强拒"] = np.where(df["审核状态"]=='机审强拒',1,0)
df["人审拒绝"] = np.where(df["审核状态"]=='人审拒绝',1,0)
df["客户取消"] = np.where(df["审核状态"]=='客户取消',1,0)
df["出库前风控强拒"] = np.where(df["审核状态"]=='出库前风控强拒',1,0)
df["无法联系"] = np.where(df["审核状态"]=='无法联系',1,0)
df["是否进件"] = np.where(df["进件"]=='进件',1,0)
df["是否出库"] = np.where(df["status"].isin([2,3,4,5,6,8,15]),1,0)

df["进件前取消"] = np.where(df["审核状态"]=='进件前取消',1,0)



In [20]:
df.columns

Index(['create_date', 'create_time', 'order_id', 'order_number', 'all_money',
       'status', 'status2', 'channel_type', 'sku_attributes', 'product_name',
       'new_actual_money', 'back_type', 'user_mobile', 'true_name',
       'id_card_num', 'total_describes', 'decision_result', 'cancel_reason',
       'trace_id', 'rejected', 'result', 'channel_name', 'activity_name',
       'merchant_id', 'merchant_name', 'total_freeze_fund_amount',
       'buy_service_product', 'service_status', 'order_method', '下单日期', '月份',
       '拒绝理由', '取消原因', '电审拒绝原因', '无法联系原因', '商品类型', '租赁方案', '押金类型', '优惠券使用否',
       '是否进行预授权', '颜色', '内存', '进件', '来源渠道', '免审', '归属渠道', '状态编码', '是否前置拦截',
       '是否机审强拒', '是否出库前风控强拒', '审核状态', '下单时段', '待审核', '前置拦截', '机审强拒', '人审拒绝',
       '客户取消', '出库前风控强拒', '无法联系', '是否进件', '是否出库', '进件前取消'],
      dtype='object')

# 取消订单

In [21]:
# 进件订单
jj_order = df.query('是否进件==1')
# 机审强拒订单
jsqj_order = df.query('机审强拒==1')
# 获取机jj_order中不在jsqj_order的订单
jsqj_unique = jj_order.index.difference(jsqj_order.index)
# 使用这些字段重新索引jj_order
jsqj_orders = jj_order.reindex(jsqj_unique)

# 出库订单
ck_order = df.query('是否出库==1')
# 获取机df_jsqj_order中不在ck_order的订单
jsqj_unique = jsqj_orders.index.difference(ck_order.index) 
# 使用这些字段重新索引df_jsqj_order
jsqj_orders = jsqj_orders.reindex(jsqj_unique) 

# # 出库前风控强拒订单
df_fkqj_order = df.query('出库前风控强拒==1')
# # 获取机jsqj_order中不在df_fkqj_order的订单
jsqj_unique = jsqj_orders.index.difference(df_fkqj_order.index) 
# # 使用这些字段重新索引df_jsqj_order
jsqj_orders = jsqj_orders.reindex(jsqj_unique) 

# # 人审拒绝订单
rs_order = df.query('人审拒绝==1')
# # 获取机jsqj_order中不在rs_order的订单
jsqj_unique = jsqj_orders.index.difference(rs_order.index) 
# # 使用这些字段重新索引df_jsqj_order
jsqj_orders = jsqj_orders.reindex(jsqj_unique) 
# jsqj_orders

order_qx = jsqj_orders.query('status2=="已退款"')
order_qx_all = jsqj_orders.query('status2=="已退款"')[['create_time', 'order_number', 'user_mobile', 'cancel_reason', 'rejected']]
order_qx_all.rename(columns={'create_time': '订单日期', 'order_number': '订单号', 'user_mobile': '手机号', 'cancel_reason': '取消原因', 'rejected': '拒绝原因'}, inplace=True)
Today1 = str(datetime.now().strftime('%Y%m%d%H'))
# with pd.ExcelWriter(f'F:/需求/取消订单数据_{Today1}.xlsx', engine='openpyxl') as writer:
#     order_qx.to_excel(writer, sheet_name='取消订单', index=False)
order_qx_all

,订单日期,订单号,手机号,取消原因,拒绝原因
3,2025-01-26 00:05:46,A202501260005454,13015699555,"88888_唐林华审核订单A202501260005454，审核结果：不通过,msg=用户无...",[]
30,2025-01-26 00:33:49,A2025012600334831,17330007604,系统操作审核订单A2025012600334831，芝麻数控-交互安全产品-风险分，审核结果...,[]
43,2025-01-26 01:04:10,A202501260104102,19527675276,系统操作审核订单A202501260104102，芝麻数控-交互安全产品-风险分，审核结果：...,[]
58,2025-01-26 01:46:36,A2025012601463619,17825936138,系统操作审核订单A2025012601463619，出库前风控审核结果：不通过,[冰鉴火眸分<500命中强拒]
59,2025-01-26 01:47:15,A2025012601471420,19825030064,"88888_黄兰娟审核订单A2025012601471420，审核结果：不通过,msg=客户...",[]
...,...,...,...,...,...
23433,2025-02-10 15:42:29,A20250210154228274,17745225259,系统操作审核订单A20250210154228274，芝麻数控-交互安全产品-风险分，审核结...,[]
23579,2025-02-10 15:58:14,A20250210155813420,18766893138,系统操作审核订单A20250210155813420，芝麻数控-交互安全产品-风险分，审核结...,[]
23631,2025-02-10 16:04:23,A2025021016042236,15953343444,"88888_黄兰娟审核订单A2025021016042236，审核结果：不通过,msg=客户...",[]
23698,2025-02-10 16:12:13,A20250210161212103,18012890566,系统操作审核订单A20250210161212103，芝麻数控-交互安全产品-风险分，审核结...,[]


In [22]:
df_rsjj = df.query('人审拒绝==1')
# df_rsjj[['create_time', 'order_number', 'cancel_reason']].

## iPhone15

In [23]:
order_qx_15 = order_qx[order_qx['product_name'].str.contains(pat=r'iPhone15|iPhone 15')]
order_qx_15['型号'] = order_qx_15['product_name'].str.extract(r'(iPhone ?15(ProMax|Pro|Plus)?)', expand=True)[0]
order_qx_15['型号内存'] = order_qx_15['型号'] + '_' + order_qx_15['内存']
order_qx_15 = order_qx_15[['create_time', 'order_number', 'user_mobile', 'cancel_reason', 'rejected', '型号内存']]
order_qx_15.rename(columns={'create_time': '下单时间', 'user_mobile': '手机号', 'order_number': '订单号', 'cancel_reason': '取消原因', 'rejected': '拒绝原因'}, inplace=True)

## iPhone16

In [24]:
order_qx_16 = order_qx[order_qx['product_name'].str.contains(pat=r'iPhone16|iPhone 16')]
order_qx_16['型号'] = order_qx_16['product_name'].str.replace(' ', '').str.extract(r'(iPhone ?16(Pro ?Max|Pro|Plus)?)', expand=True)[0]
order_qx_16['型号内存'] = order_qx_16['型号'] + '_' + order_qx_16['内存']
order_qx_16 = order_qx_16[['create_time', 'order_number', 'user_mobile', 'cancel_reason', 'rejected', '型号内存']]
order_qx_16.rename(columns={'create_time': '下单时间', 'order_number': '订单号', 'user_mobile':'手机号','cancel_reason': '取消原因', 'rejected': '拒绝原因'}, inplace=True)

## 华为

In [25]:
def HuaWei(df):
    df = df[df['product_name'].str.contains(r'华为')]
    # 定义更完善的正则表达式模式
    pattern = re.compile(r'(华为 ?(Mate|Pura|畅享|P|nova) ?\d+ ?(Pro\+|Pro|Plus\+|Plus|Flip\+|Flip)?|nova ?\d+ ?(Pro\+|Pro|Ultra\+|Ultra|Flip\+|Flip)?|华为 ?nova ? Flip?)')
    # 使用 apply 和 lambda 函数匹配描述中的手机型号
    df['型号'] = df['product_name'].apply(lambda x: pattern.findall(x))
    # 将匹配结果展平为单个字符串
    df['型号'] = df['型号'].apply(lambda x: [item[0] for item in x])
    # 过滤出包含手机型号的行
    df_filtered = df[df['型号'].apply(lambda x: len(x) > 0)]
    # 展开列表，确保每个型号都在单独的一行中
    df_expanded = df_filtered.explode('型号')
    df_expanded['型号'] = df_expanded['型号'].str.replace(' ', '')

    df_expanded['型号内存'] = df_expanded['型号']+'_'+df_expanded['内存']
    
    return df_expanded



In [26]:
huawei = HuaWei(order_qx)
huawei = huawei[huawei['型号'].str.contains('华为Pura')]
huawei = huawei[['create_time', 'order_number', 'user_mobile', 'cancel_reason', 'rejected', '型号内存']]
huawei.rename(columns={'create_time': '下单时间', 'order_number': '订单号', 'user_mobile':'手机号', 'cancel_reason': '取消原因', 'rejected': '拒绝原因'}, inplace=True)
huawei
# with pd.ExcelWriter('')

,下单时间,订单号,手机号,取消原因,拒绝原因,型号内存
11184,2025-02-06 15:00:01,A202502061500001,15100490147,系统操作审核订单A202502061500001，芝麻数控-交互安全产品-风险分，审核结果：...,[],华为Pura70Pro+_16GB+512GB
18334,2025-02-08 18:09:37,A2025020818093640,18352012301,"88888_谢金凤审核订单A2025020818093640，审核结果：不通过,msg=客户...",[],华为Pura70_12GB+512GB


## 芝麻活动

In [27]:

order_qx_zm = order_qx[order_qx['product_name'].str.contains(pat=r'芝麻活动专享3')]
order_qx_zm['型号'] = order_qx_zm['product_name'].str.replace(' ', '').str.extract(r'(iPhone ?16(Pro ?Max|Pro|Plus)?|iPhone ?15(Pro ?Max|Pro|Plus)?)', expand=True)[0]
order_qx_zm['活动名称'] = order_qx_zm['product_name'].str.extract(r'(芝麻活动专享3?)', expand=True)[0]
order_qx_zm['型号内存'] = order_qx_zm['型号'] + '_' + order_qx_zm['内存']
order_qx_zm = order_qx_zm[['create_time', 'order_number', 'cancel_reason', 'rejected', '型号内存', '活动名称']]
order_qx_zm.rename(columns={'create_time': '下单时间', 'order_number': '订单号', 'cancel_reason': '取消原因', 'rejected': '拒绝原因'}, inplace=True)

In [28]:
df_rsjj = df[df['人审拒绝']==1]
df_rsjj = df_rsjj[['create_time', 'order_number', 'user_mobile', '电审拒绝原因']]
df_rsjj.rename(columns={'create_time': '订单日期', 'order_number': '订单号', 'user_mobile': '手机号'}, inplace=True)
df_rsjj
# with pd.ExcelWriter(f'F:/需求/取消订单数据_{Today1}.xlsx', engine='openpyxl', mode='a') as writer:
#     df_rsjj.to_excel(writer, sheet_name='人审拒绝', index=False)

,订单日期,订单号,手机号,电审拒绝原因
0,2025-01-26 00:01:25,A202501260001251,18659161147,"拒绝电审""]"
234,2025-01-26 10:01:20,A202501261001192,18953165484,"拒绝电审""]"
341,2025-01-26 11:44:45,A2025012611444447,13657936605,"短期在租多台""]"
1253,2025-01-27 11:09:12,A202501271109129,13855944446,"拒绝电审""]"
1434,2025-01-27 15:02:08,A202501271502073,15369618688,"第三方指导电审""]"
...,...,...,...,...
23409,2025-02-10 15:39:15,A20250210153915250,19527528545,None
23551,2025-02-10 15:55:12,A20250210155511393,18947889779,None
23696,2025-02-10 16:12:03,A20250210161203101,18214559878,None
23734,2025-02-10 16:16:31,A20250210161631139,15245768181,None


In [ ]:
df_jjsum = df[df['是否进件']==1]['是否进件'].sum()
df_cksum = df[df['是否出库']==1]['是否出库'].sum()
df_cksum

In [ ]:
df_zm = df[df['product_name'].str.contains(pat=r'芝麻活动专享3')]
df_zm_jjsum = df_zm[df_zm['是否进件']==1]['是否进件'].sum()
df_zm_cksum = df_zm[df_zm['是否出库']==1]['是否出库'].sum()
df_zm_1 = df_zm[df_zm['product_name'].str.contains(pat='首月1元')]
# df_zm_1_count = df_zm_1[df_zm_1['是否进件']==1].count()
df_zm_1_jjsum = df_zm_1[df_zm_1['是否进件']==1]['是否进件'].sum()
df_zm_1_cksum = df_zm_1[df_zm_1['是否出库']==1]['是否出库'].sum()
df_zm_1_cksum

In [ ]:
df_zm_result = pd.DataFrame({
    '总进件': [df_jjsum],
    '总出库': [df_cksum],
    '活动进件': [df_zm_jjsum],
    '活动出库': [df_zm_cksum]
})
df_zm_result['活动进件占比'] = df_zm_jjsum/df_jjsum * 100
df_zm_result['活动进件占比'] = df_zm_result['活动进件占比'].apply(lambda x: f'{x:.2f}%')

df_zm_result['活动出库占比'] = df_zm_cksum/df_cksum * 100
df_zm_result['活动出库占比'] = df_zm_result['活动出库占比'].apply(lambda x: f'{x:.2f}%')


df_zm_result
# with pd.ExcelWriter('F:/需求/芝麻活动进件占比.xlsx', engine='openpyxl') as writer:
#     df_zm_result.to_excel(writer, sheet_name='活动进件占比', index=False)

## 取消原因汇总

In [ ]:
order_qx['product_name'] = order_qx['product_name'].str.replace(' ', '')
order_qx_iPhone = order_qx[order_qx['product_name'].str.contains(r'iPhone15|iPhone 15|iPhone16|iPhone 16')]
order_qx_iPhone['型号'] = order_qx_iPhone['product_name'].str.extract(r'(iPhone ?15(Pro ?Max|Pro|Plus)?|iPhone ?16(Pro ?Max|Pro|Plus)?)', expand=True)[0]
order_qx_iPhone['型号内存'] = order_qx_iPhone['型号'] + '_' + order_qx_iPhone['内存']
order_qx_iPhone['取消原因2'] = order_qx_iPhone['cancel_reason'].str.split('：')
order_qx_iPhone['取消原因2'] = order_qx_iPhone['取消原因2'].apply(lambda x: x[-1] if x is not None else x)
order_qx_iPhone = order_qx_iPhone[['create_date', 'order_number', '型号内存', '取消原因2']]
order_qx_iPhone.rename(columns={'create_date': '下单时间', 'order_number': '订单号'}, inplace=True)
order_qx_iPhone_group1 = order_qx_iPhone.groupby('下单时间').agg({'取消原因2': 'count'})
order_qx_iPhone_group1.rename(columns={'取消原因2': '日总数量'}, inplace=True)
order_qx_iPhone_group1 = order_qx_iPhone_group1.reset_index()
order_qx_iPhone_group2 = order_qx_iPhone.groupby(['下单时间', '型号内存', '取消原因2']).agg({'取消原因2': 'count'})
order_qx_iPhone_group2.rename(columns={'取消原因2': '数量'}, inplace=True)
order_qx_iPhone_group2 = order_qx_iPhone_group2.reset_index()
# order_qx_iPhone_group3 = order_qx_iPhone_group2.groupby(['下单时间', '型号内存'])[['数量']].cumsum().last()
order_qx_iPhone_group = pd.merge(order_qx_iPhone_group1, order_qx_iPhone_group2, on='下单时间', how='left')
order_qx_iPhone_group['比列'] = order_qx_iPhone_group['数量']/order_qx_iPhone_group['日总数量'] * 100
order_qx_iPhone_group['比列'] = order_qx_iPhone_group['比列'].apply(lambda x: f'{x:.2f}%')
order_qx_iPhone_group = order_qx_iPhone_group[['下单时间', '型号内存', '取消原因2', '数量', '日总数量', '比列']]
# order_qx_iPhone_group
qx_path = r'\\digua\迪瓜租机\13.运营-信审每日数据/'
with pd.ExcelWriter(qx_path + f'机型取消原因_{Today1}.xlsx', engine='openpyxl') as writer:
    order_qx_iPhone_group.to_excel(writer, sheet_name='机型取消原因', index=False)

In [ ]:
huawei['下单日期'] = huawei['下单时间'].dt.strftime('%Y-%m-%d')
huawei['取消原因2'] = huawei['取消原因'].str.split('：')
huawei['取消原因2'] = huawei['取消原因2'].apply(lambda x: x[-1] if x is not None else x)
huawei['取消原因2'].drop_duplicates()
huawei1 = huawei.groupby('下单日期').agg({'取消原因2': 'count'})
huawei1.rename(columns={'取消原因2': '日总数量'}, inplace=True)
huawei1 = huawei1.reset_index()
huawei2 = huawei.groupby(['下单日期', '型号内存', '取消原因2']).agg({'取消原因2': 'count'})
huawei2.rename(columns={'取消原因2': '数量'}, inplace=True)
huawei2 = huawei2.reset_index()
huawei_group = pd.merge(huawei1, huawei2, on='下单日期', how='left')
huawei_group['比列'] = huawei_group['数量']/huawei_group['日总数量'] * 100
huawei_group['比列'] = huawei_group['比列'].apply(lambda x: f'{x:.2f}%')
huawei_group = huawei_group[['下单日期', '型号内存', '取消原因2', '数量', '日总数量', '比列']]
huawei_group
with pd.ExcelWriter(qx_path + f'机型取消原因_{Today1}.xlsx', engine='openpyxl', mode='a') as writer:
    huawei_group.to_excel(writer, sheet_name='华为机型取消原因', index=False)

# 设置定时任务

In [ ]:
from apscheduler.schedulers.blocking import BlockingScheduler
from apscheduler.schedulers.background import BackgroundScheduler

# hour = 18
# minute = 00
week_name = pd.Timestamp.now().dayofweek
H = datetime.now().strftime('%H')
M = datetime.now().strftime('%M')
def my_job1():
    # print(f'执行定时任务：现在是每日的{hour}点{minute}分')
    print(f'执行定时任务：现在是每日的{H}点{int(M)+1}分')
    # path1 = r'\\digua\迪瓜租机\13.运营-信审每日数据/' # 共享盘 \\digua
    path1 = 'F:/需求/取消订单/'
    Today1 = str(datetime.now().strftime('%Y%m%d%H'))
    with pd.ExcelWriter(path1 + f'取消订单数据1_{Today1}.xlsx', engine='openpyxl') as writer:
        order_qx_all.to_excel(writer, sheet_name='取消订单', index=False)
    with pd.ExcelWriter(path1 + f'取消订单数据1_{Today1}.xlsx', engine='openpyxl', mode='a') as writer:
        order_qx_15.to_excel(writer, sheet_name='iPhone15', index=False)
        order_qx_16.to_excel(writer, sheet_name='iPhone16', index=False)
        huawei.to_excel(writer, sheet_name='华为', index=False)
        order_qx_zm.to_excel(writer, sheet_name='芝麻活动', index=False)
        df_rsjj.to_excel(writer, sheet_name='电审拒绝', index=False)
scheduler1 = BackgroundScheduler()
# 添加一个cron任务，每天的9:00执行
# scheduler1.add_job(my_job1, 'cron', hour=hour, minute=minute)
scheduler1.add_job(my_job1, 'cron', hour=H, minute=int(M)+1)

# 启动scheduler1
scheduler1.start()
    

In [ ]:
# path1 = 'F:/' # 共享盘 \\digua
#     # path1 = 'F:/需求/'
# Today1 = str(datetime.now().strftime('%Y%m%d%H'))
# with pd.ExcelWriter(path1 + f'取消订单数据_{Today1}.xlsx', engine='openpyxl') as writer:
#     order_qx_all.to_excel(writer, sheet_name='取消订单', index=False)
# with pd.ExcelWriter(path1 + f'取消订单数据_{Today1}.xlsx', engine='openpyxl', mode='a') as writer:
#     order_qx_15.to_excel(writer, sheet_name='iPhone15', index=False)
#     order_qx_16.to_excel(writer, sheet_name='iPhone16', index=False)
#     huawei.to_excel(writer, sheet_name='华为', index=False)
#     order_qx_zm.to_excel(writer, sheet_name='芝麻活动', index=False)
#     df_rsjj.to_excel(writer, sheet_name='电审拒绝', index=False)